In [1]:
import pandas as pd
import numpy as np
import unicodedata
import string
import re
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle
import random
import pdb
from torch.utils.data import DataLoader
import logging
import itertools
import unicodedata
import string
import re
import random
import argparse
from torch import optim
import time
import os
from bleu_score import BLEU_SCORE
# from models_viet import EncoderRNN, AttentionDecoderRNN, DecoderRNN
from load_dataset_viet import *
# from define_training_viet import *
import torchtext
from torch.utils.data import BatchSampler
from torch.utils.data import SequentialSampler
from torch.utils.data import Sampler
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
device = torch.device('cuda')

In [3]:
class Vietnamese(Dataset):
    def __init__(self, df, val = False):
        self.df = df
        self.val = val
        
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        english = self.df.iloc[idx,:]['en_idized']
        viet = self.df.iloc[idx,:]['vi_idized']
        en_len = self.df.iloc[idx,:]['en_len']
        vi_len = self.df.iloc[idx,:]['vi_len']
        if self.val:
            en_data = self.df.iloc[idx,:]['en_data'].lower()
            return [viet,english,vi_len,en_len,en_data]
        else:
            return [viet,english,vi_len,en_len]
    
    
def vocab_collate_func(batch):
    MAX_LEN_EN = 48
    MAX_LEN_VI = 48
    en_data = []
    vi_data = []
    en_len = []
    vi_len = []
    for datum in batch:
        en_len.append(datum[3])
        vi_len.append(datum[2])
    max_batch_length_en = max(en_len)
    max_batch_length_vi = max(vi_len)
    if max_batch_length_en < MAX_LEN_EN:
        MAX_LEN_EN = max_batch_length_en
    if max_batch_length_vi < MAX_LEN_VI:
        MAX_LEN_VI = max_batch_length_vi
    # padding
    for datum in batch:
        if datum[2]>MAX_LEN_VI:
            padded_vec_s1 = np.array(datum[0])[:MAX_LEN_VI]
        else:
            padded_vec_s1 = np.pad(np.array(datum[0]),
                                pad_width=((0,MAX_LEN_VI - datum[2])),
                                mode="constant", constant_values=PAD_IDX)
        if datum[3]>MAX_LEN_EN:
            padded_vec_s2 = np.array(datum[1])[:MAX_LEN_EN]
        else:
            padded_vec_s2 = np.pad(np.array(datum[1]),
                                pad_width=((0,MAX_LEN_EN - datum[3])),
                                mode="constant", constant_values=PAD_IDX)
        en_data.append(padded_vec_s2)
        vi_data.append(padded_vec_s1)
    vi_data = np.array(vi_data)
    en_data = np.array(en_data)
    vi_len = np.array(vi_len)
    en_len = np.array(en_len)
    sorted_vi_len = np.argsort(-vi_len)
    vi_data = vi_data[sorted_vi_len]
    en_data = en_data[sorted_vi_len]
    vi_len = vi_len[sorted_vi_len]
    en_len = en_len[sorted_vi_len]
#     print(en_len)
    vi_len[vi_len>MAX_LEN_VI] = MAX_LEN_VI
    en_len[en_len>MAX_LEN_EN] = MAX_LEN_EN
        
    return [torch.from_numpy(vi_data), torch.from_numpy(en_data),
            torch.from_numpy(vi_len), torch.from_numpy(en_len)]

def convert_idx_2_sent(tensor, lang_obj):
    word_list = []
    for i in tensor:
        if i.item() not in set([PAD_IDX,EOS_token,SOS_token]):
            word_list.append(lang_obj.index2word[i.item()])
    return (' ').join(word_list)

def convert_id_list_2_sent(list_idx, lang_obj):
    word_list = []
    if type(list_idx) == list:
        for i in list_idx:
            if i not in set([EOS_token]):
                word_list.append(lang_obj.index2word[i])
    else:
        for i in list_idx:
            if i.item() not in set([EOS_token,SOS_token,PAD_IDX]):
                word_list.append(lang_obj.index2word[i.item()])
    return (' ').join(word_list)

In [4]:
def vocab_collate_func_val(batch):
    return [torch.from_numpy(np.array(batch[0][0])).unsqueeze(0), torch.from_numpy(np.array(batch[0][1])).unsqueeze(0),
            torch.from_numpy(np.array(batch[0][2])).unsqueeze(0), torch.from_numpy(np.array(batch[0][3])).unsqueeze(0),batch[0][4]]

In [5]:
# MAX_LEN = 48
train,val,en_lang,vi_lang = train_val_load(48, "", '/scratch/ark576/machine_translation_data/')

In [6]:
def shuffle_sorted_batches(train, bs):
    batch_samp_list = list(BatchSampler(SequentialSampler(train), bs, drop_last = False))
    np.random.shuffle(batch_samp_list)
    batch_samp_list_merged = list(itertools.chain(*batch_samp_list))
    return train.iloc[batch_samp_list_merged,:]

In [7]:
# train_sorted = train.sort_values('en_len', ascending = False)
bs_dict = {'train':128,'validate':1, 'train_val':1}
# train_used = train_sorted
train_used = train.sample(10)
# train_used = shuffle_sorted_batches(train_sorted,bs_dict['train'])
collate_fn_dict = {'train':vocab_collate_func, 'validate':vocab_collate_func_val, 'train_val':vocab_collate_func_val}
transformed_dataset = {'train': Vietnamese(train_used),
                       'validate': Vietnamese(val, val = True),
                       'train_val':Vietnamese(train.iloc[:50], val = True)
                                               }

dataloader = {x: DataLoader(transformed_dataset[x], batch_size=bs_dict[x], collate_fn=collate_fn_dict[x],
                    shuffle=False, num_workers=0) for x in ['train', 'validate', 'train_val']}

In [8]:
# train.head()

In [9]:
train_data_sample = next(iter(dataloader['train']))

In [10]:
train_data_sample[0].shape

torch.Size([10, 48])

In [11]:
train_data_sample[1].shape

torch.Size([10, 45])

In [12]:
train_data_sample[2]

tensor([48, 46, 25, 19, 15, 13, 10,  8,  8,  7])

In [13]:
train_data_sample[3]

tensor([45, 44, 27, 24, 14, 14, 11,  7,  5,  6])

In [14]:
for t in zip(train_data_sample[1],train_data_sample[0]):
    en_sent = []
    vi_sent = []
    for i in t[0]:
        en_sent.append(en_lang.index2word[i.item()])
    for i in t[1]:
        vi_sent.append(vi_lang.index2word[i.item()])
    print('English:',(' ').join(en_sent))
    print('Viet:',(' ').join(vi_sent))
    print('-*'*50)

English: so , 5,000 years after the invention of the wheel , we have a new wheel . and i will show you , in the next video -- can you start it , please ? -- that very heavy loads can be moved . EOS
Viet: vì_thế 5000 năm sau khi bánh_xe được phát_minh , chúng_ta lại có một loại bánh_xe mới . tôi sẽ cho các bạn xem trong đoạn phim kế_tiếp - - - anh có_thể làm_ơn bật nó lên được không ? - - mà những vật rất nặng_nề có_thể di_chuyển
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
English: and the reason the british were coming was because they would not treat anybody with anything under age 16 , which means they were UKN them to an adult body , no matter what happened , even if they tested them well . EOS PAD
Viet: lý_do người_ta gửi cháu sang đây là bên đó người_ta không điều_trị giới cho ai dưới 16 tuổi , có nghĩa họ cho việc đó chỉ được làm trong cơ_thể người_lớn , bất_chấp cái_gì đang xảy ra , cả khi xét_nghiệm cho thấy rõ_ràng . EOS

In [15]:
vi_lang.n_words

13294

In [16]:
en_lang.n_words

16031

In [17]:
# class OldEncoderRNN(nn.Module):
#     def __init__(self, input_size, hidden_size,bi):
#         super(OldEncoderRNN, self).__init__()
#         self.bi=bi
#         if self.bi:
#             self.mul=2
#         else:
#             self.mul=1
#         self.hidden_size = hidden_size

#         self.embedding = nn.Embedding(input_size, hidden_size)
#         self.gru = nn.GRU(hidden_size, hidden_size,batch_first=True,bidirectional=self.bi)

#     def forward(self, input, hidden):
#         embedded = self.embedding(input)
#         output = embedded
#         output, hidden = self.gru(output, hidden)
#         last_layer = output[:,-1,:self.hidden_size]
#         first_layer = output[:,0,self.hidden_size:]
#         hidden = torch.cat([first_layer,last_layer],dim=1).unsqueeze(0)
#         return output, hidden

#     def initHidden(self,bs):
#         return torch.zeros(self.mul, bs, self.hidden_size).to(device)
    
    
# class OldAttentionDecoderRNN(nn.Module):
#     def __init__(self, hidden_size, output_size,bi, MAX_LEN,n_layers = 1, attention_type = None):
#         super(OldAttentionDecoderRNN, self).__init__()
#         self.mul=2
#         self.hidden_size = hidden_size

#         self.embedding = nn.Embedding(output_size, hidden_size)
#         self.dropout = nn.Dropout(p=0.2)
#         self.gru = nn.GRU(hidden_size*2 + hidden_size, hidden_size*2,batch_first=True, num_layers = n_layers)
        
#         self.attention_type = attention_type
#         if self.attention_type is not None:
#             self.attn = nn.Linear(self.hidden_size*2, self.hidden_size*2)
#             self.attn_drop = nn.Dropout(p = 0.5)
#         else:
#             self.attn = nn.Linear(self.hidden_size * 2, MAX_LEN)
        
# #         self.attn_combine = nn.Linear(self.hidden_size * self.mul+self.hidden_size, self.hidden_size)
        
#         self.out = nn.Linear(self.mul*hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)

#     def forward(self, input, hidden,encoder_outputs):
#         bss = input.size(0)
#         output = self.embedding(input)
#         output = self.dropout(output)
#         att_out = self.attn_drop(self.attn(hidden[-1])).unsqueeze(-1)
#         if self.attention_type is not None:
#             attn_wts = F.softmax(torch.bmm(encoder_outputs,att_out),dim = 1)
#             attn_applied = torch.bmm(encoder_outputs.transpose(1,2),attn_wts).transpose(1,2)
#         else:
#             att_out = F.softmax(self.attn(cat),dim=1)
#             attn_applied = torch.bmm(att_out,encoder_outputs)
#         attn_cat = torch.cat((output, attn_applied), 2)
#         output = F.relu(attn_cat)
#         output, hidden = self.gru(output, hidden)
#         output = self.out(output.squeeze(dim=1))
#         output = self.softmax(output)

#         return output, hidden

#     def initHidden(self):
#         return torch.zeros(self.mul, bs, self.hidden_size).to(device)

In [18]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embed_dim, hidden_size,n_layers, rnn_type = 'lstm'):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = Embedding(input_size, embed_dim, PAD_IDX)
        self.rnn_type =  rnn_type
        self.dropout_in = nn.Dropout(p = 0.1)
        self.n_layers = n_layers
        if rnn_type == 'gru':
            self.rnn = nn.GRU(embed_dim, hidden_size,batch_first=True,bidirectional=True, num_layers = self.n_layers, dropout = 0.2)
        elif rnn_type == 'lstm':
            self.rnn = LSTM(embed_dim, hidden_size, batch_first=True,bidirectional=True, num_layers = n_layers,dropout = 0.2)

    def forward(self, input, src_len):
        embedded = self.embedding(input)
        bs = embedded.size(0)
        output = self.dropout_in(embedded)
#         print("encoded Input", output.shape)
#         output = torch.nn.utils.rnn.pack_padded_sequence(output, src_len, batch_first=True)
        if self.rnn_type == 'gru':
            hidden =  self.initHidden(bs)
            output, hidden = self.rnn(output, hidden)
#             output = torch.nn.utils.rnn.pad_packed_sequence(output, batch_first=True)
        elif self.rnn_type == 'lstm':
            hidden, c = self.initHidden(bs)
            packed_output = nn.utils.rnn.pack_padded_sequence(output, src_len.data.tolist(), batch_first = True)
            packed_outs, (hiddden, c) = self.rnn(packed_output,(hidden, c))
            output, _ = nn.utils.rnn.pad_packed_sequence(packed_outs, padding_value=PAD_IDX, batch_first = True)
        hidden = hidden.view(self.n_layers, 2, bs, -1).transpose(1, 2).contiguous().view(self.n_layers, bs, -1)
        c = c.view(self.n_layers, 2, bs, -1).transpose(1, 2).contiguous().view(self.n_layers, bs, -1)
#         print(output.shape)
#         print(hidden.shape)
#         print(c.shape)
#             output = torch.nn.utils.rnn.pad_packed_sequence(output,batch_first=True)
#         print(hidden.shape)
#         print("rnn output", output.shape)
#         print("rnn hidden", hidden.shape)
#         hidden = hidden.transpose(0,1).contiguous().view(bs,-1)
#         hidden_1 = torch.tanh(self.linear1(hidden)).unsqueeze(0)
# #         print("rnn hidden_1", hidden_1.shape)
#         hidden_2 = torch.tanh(self.linear2(hidden)).unsqueeze(0)
# #         print("rnn hidden_2", hidden_2.shape)
#         hidden = torch.cat([hidden_1,hidden_2],dim = 0)
#         print("rnn hidden_final", hidden.shape)
#             print(hidden.shape)
#         last_layer = output[:,-1,:self.hidden_size]
#         first_layer = output[:,0,self.hidden_size:]
#         hidden = first_layer + last_layer
        return output, hidden, c

    def initHidden(self,bs):
        if self.rnn_type == 'gru' :
            return torch.zeros(self.n_layers*2, bs, self.hidden_size).to(device)
        elif self.rnn_type == 'lstm':
            return torch.zeros(self.n_layers*2,bs,self.hidden_size).to(device),torch.zeros(self.n_layers*2,bs,self.hidden_size).to(device)

class Attention_Module(nn.Module):
    def __init__(self, hidden_dim, output_dim):
        super(Attention_Module, self).__init__()
        self.l1 = Linear(hidden_dim, output_dim, bias = False)
        self.l2 = Linear(hidden_dim+output_dim, output_dim, bias =  False)
        
    def forward(self, hidden, encoder_outs, src_lens):
        ''' hiddden: bsz x hidden_dim
        encoder_outs: bsz x sq_len x encoder dim (output_dim)
        src_lens: bsz
        
        x: bsz x output_dim
        attn_score: bsz x sq_len'''
        x = self.l1(hidden)
        att_score = (encoder_outs.transpose(0,1) * x.unsqueeze(0)).sum(dim = 2)
        seq_mask = sequence_mask(src_lens, max_len = max(src_lens).item()).transpose(0,1)
        masked_att = seq_mask*att_score
        masked_att[masked_att==0] = -1e10
        attn_scores = F.softmax(masked_att, dim=0)
        x = (attn_scores.unsqueeze(2) * encoder_outs.transpose(0,1)).sum(dim=0)
        x = torch.tanh(self.l2(torch.cat((x, hidden), dim=1)))
        return x, attn_scores
        
class AttentionDecoderRNN(nn.Module):
    def __init__(self, output_size, embed_dim, hidden_size, n_layers = 1, attention = False):
        super(AttentionDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        encoder_output_size = hidden_size
        self.embedding = Embedding(output_size, embed_dim, PAD_IDX)
        self.dropout = nn.Dropout(p=0.1)
        self.n_layers = n_layers
        self.att_layer = Attention_Module(self.hidden_size, encoder_output_size) if attention else None
        self.layers = nn.ModuleList([
            LSTMCell(
                input_size=self.hidden_size + embed_dim if layer == 0 else hidden_size,
                hidden_size=hidden_size,
            )
            for layer in range(self.n_layers)
        ])
#         if self.rnn_type == 'gru':
#             self.rnn = nn.GRU(self.hidden_size + embed_dim, hidden_size,batch_first=True, num_layers = self.n_layers, )
#         elif self.rnn_type == 'lstm':
#             self.rnn = nn.LSTM(self.hidden_size + embed_dim, hidden_size,batch_first=True, num_layers = self.n_layers)
        self.fc_out = nn.Linear(self.hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        

    def forward(self, input,context_vector, prev_hiddens,prev_cs,encoder_outputs,src_len):
        bsz = input.size(0)
        output = self.embedding(input)
        output = self.dropout(output)
#         print("decoder Input embedded", output.shape)
        cated_input = torch.cat([output.squeeze(1),context_vector], dim = 1)
#         print("cated_input",cated_input.shape)
        new_hiddens = []
        new_cs = []
        for i, rnn in enumerate(self.layers):
            hidden, c = rnn(cated_input, (prev_hiddens[i], prev_cs[i]))
            cated_input = self.dropout(hidden)
            new_hiddens.append(hidden.unsqueeze(0))
            new_cs.append(c.unsqueeze(0))
        new_hiddens = torch.cat(new_hiddens, dim = 0)
        new_cs = torch.cat(new_cs, dim = 0)

        # apply attention using the last layer's hidden state
        if self.att_layer is not None:
            out, attn_score = self.att_layer(hidden, encoder_outputs, src_len)
        else:
            out = hidden
            attn_score = None
        out = self.dropout(out)
        out_vocab = self.softmax(self.fc_out(out))

        return out_vocab, out, new_hiddens, new_cs, attn_score
    
#     def initHidden(self,bs):
#         if self.rnn_type == 'gru' :
#             return None
#         elif self.rnn_type == 'lstm':
#             return torch.zeros(self.n_layers,bs,self.hidden_size).to(device)

In [20]:
def Embedding(num_embeddings, embedding_dim, padding_idx):
    m = nn.Embedding(num_embeddings, embedding_dim, padding_idx=padding_idx)
    nn.init.uniform_(m.weight, -0.1, 0.1)
    nn.init.constant_(m.weight[padding_idx], 0)
    return m


def LSTM(input_size, hidden_size, **kwargs):
    m = nn.LSTM(input_size, hidden_size,**kwargs)
    for name, param in m.named_parameters():
        if 'weight' in name or 'bias' in name:
            param.data.uniform_(-0.1, 0.1)
    return m


def LSTMCell(input_size, hidden_size, **kwargs):
    m = nn.LSTMCell(input_size, hidden_size,**kwargs)
    for name, param in m.named_parameters():
        if 'weight' in name or 'bias' in name:
            param.data.uniform_(-0.1, 0.1)
    return m


def Linear(in_features, out_features, bias=True, dropout=0):
    """Linear layer (input: N x T x C)"""
    m = nn.Linear(in_features, out_features, bias=bias)
    m.weight.data.uniform_(-0.1, 0.1)
    if bias:
        m.bias.data.uniform_(-0.1, 0.1)
    return m

In [21]:
def sequence_mask(sequence_length, max_len=None):
    if max_len is None:
        max_len = sequence_length.max().item()
    batch_size = sequence_length.size(0)
    seq_range = torch.arange(0, max_len).long()
    seq_range_expand = seq_range.unsqueeze(0).repeat([batch_size,1])
    seq_range_expand = seq_range_expand.to(device)
    seq_length_expand = (sequence_length.unsqueeze(1)
                         .expand_as(seq_range_expand))
    return (seq_range_expand < seq_length_expand).float()

In [22]:
# def encode_decode(encoder,decoder,data_en,data_de,src_len,tar_len,m_type,rand_num = 0.5):
#     use_teacher_forcing = True if random.random() < rand_num else False
# #     print("tar_len",tar_len)
#     bss = data_en.size(0)
#     en_hid = encoder.initHidden(bss)
#     en_out,en_hid = encoder(data_en, en_hid)
#     max_tar_len_batch = max(tar_len).item()
#     decoder_hidden = en_hid
#     decoder_input = torch.tensor([[SOS_token]]*bss).to(device)
# #     c = decoder.initHidden(bss)
# #     print("max_tar_len_batch",max_tar_len_batch)
#     if use_teacher_forcing:
#         d_out = []
#         for i in range(max_tar_len_batch):
#             if m_type=="attention":
#                 decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden,en_out)   
#             else:
#                 decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden)
#             d_out.append(decoder_output.unsqueeze(-1))
#             decoder_input = data_de[:,i].view(-1,1)
#         d_hid = decoder_hidden
#         d_out = torch.cat(d_out,dim=-1)
#     else:
#         d_out = []
#         for i in range(max_tar_len_batch):
#             if m_type=="attention":
#                 decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden,en_out)
#             else:
#                 decoder_output, decoder_hidden = decoder(decoder_input,decoder_hidden)
#             d_out.append(decoder_output.unsqueeze(-1))
#             topv, topi = decoder_output.topk(1)
# #             decoder_input = topi.squeeze().detach().view(-1,1)
#             decoder_input = topi.squeeze().view(-1,1)
#         d_hid = decoder_hidden
#         d_out = torch.cat(d_out,dim=-1)
#     return d_out, d_hid

In [23]:
# def encode_decode(encoder,decoder,data_en,data_de,src_len,tar_len,m_type,rand_num = 0.5):
#     use_teacher_forcing = True if random.random() < rand_num else False
# #     print("tar_len",tar_len)
#     bss = data_en.size(0)
#     en_out,en_hid,c = encoder(data_en, src_len)
#     max_tar_len_batch = max(tar_len).item()
#     decoder_hidden = en_hid
#     decoder_input = torch.tensor([[SOS_token]]*bss).to(device)
# #     print("max_tar_len_batch",max_tar_len_batch)
#     if use_teacher_forcing:
#         d_out = []
#         for i in range(max_tar_len_batch):
#             if m_type=="attention":
#                 decoder_output,decoder_hidden, c = decoder(decoder_input,decoder_hidden,en_out,src_len,c)   
#             else:
#                 decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden)
#             d_out.append(decoder_output.unsqueeze(-1))
#             decoder_input = data_de[:,i].view(-1,1)
#         d_hid = decoder_hidden
#         d_out = torch.cat(d_out,dim=-1)
# #         print("softmax mat", d_out.shape)
#     else:
#         d_out = []
#         for i in range(max_tar_len_batch):
#             if m_type=="attention":
#                 decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden,en_out,src_len,c)
#             else:
#                 decoder_output, decoder_hidden = decoder(decoder_input,decoder_hidden)
#             d_out.append(decoder_output.unsqueeze(-1))
#             topv, topi = decoder_output.topk(1)
# #             decoder_input = topi.squeeze().detach().view(-1,1)
#             decoder_input = topi.squeeze().view(-1,1)
#         d_hid = decoder_hidden
#         d_out = torch.cat(d_out,dim=-1)
#     return d_out, d_hid

In [24]:
def encode_decode(encoder,decoder,data_en,data_de,src_len,tar_len,rand_num = 0.95, val = False):
    if not val:
        use_teacher_forcing = True if random.random() < rand_num else False
    #     print("tar_len",tar_len)
        bss = data_en.size(0)
        en_out,en_hid,en_c = encoder(data_en, src_len)
        max_src_len_batch = max(src_len).item()
        max_tar_len_batch = max(tar_len).item()
        prev_hiddens = en_hid
        prev_cs = en_c
        decoder_input = torch.tensor([[SOS_token]]*bss).to(device)
        prev_output = torch.zeros((bss, en_out.size(-1))).to(device)
        if use_teacher_forcing:
            d_out = []
            for i in range(max_tar_len_batch):
                out_vocab, prev_output,prev_hiddens, prev_cs, attention_score = decoder(decoder_input,prev_output, \
                                                                                        prev_hiddens,prev_cs, en_out,\
                                                                                        src_len)
                d_out.append(out_vocab.unsqueeze(-1))
                decoder_input = data_de[:,i].view(-1,1)
            d_out = torch.cat(d_out,dim=-1)
    #         print("softmax mat", d_out.shape)
        else:
            d_out = []
            for i in range(max_tar_len_batch):
                out_vocab, prev_output,prev_hiddens, prev_cs, attention_score = decoder(decoder_input,prev_output, \
                                                                                        prev_hiddens,prev_cs, en_out,\
                                                                                        src_len)
                d_out.append(out_vocab.unsqueeze(-1))
                topv, topi = out_vocab.topk(1)
                decoder_input = topi.squeeze().detach().view(-1,1)
    #             decoder_input = topi.squeeze().view(-1,1)
            d_out = torch.cat(d_out,dim=-1)
        return d_out
    else:
        encoder.eval()
        decoder.eval()
        bss = data_en.size(0)
        en_out,en_hid,en_c = encoder(data_en, src_len)
        max_src_len_batch = max(src_len).item()
        max_tar_len_batch = max(tar_len).item()
        prev_hiddens = en_hid
        prev_cs = en_c
        decoder_input = torch.tensor([[SOS_token]]*bss).to(device)
        prev_output = torch.zeros((bss, en_out.size(-1))).to(device)
        d_out = []
        for i in range(max_tar_len_batch):
            out_vocab, prev_output,prev_hiddens, prev_cs, attention_score = decoder(decoder_input,prev_output, \
                                                                                    prev_hiddens,prev_cs, en_out,\
                                                                                    src_len)
            d_out.append(out_vocab.unsqueeze(-1))
            topv, topi = out_vocab.topk(1)
            decoder_input = topi.squeeze().detach().view(-1,1)
#             decoder_input = topi.squeeze().view(-1,1)
        d_out = torch.cat(d_out,dim=-1)
        return d_out

In [25]:
def flatten_cel_loss(input,target,nll):
    input = input.transpose(1,2)
    bs, sl = input.size()[:2]
    return nll(input.contiguous().view(bs*sl,-1),target.contiguous().view(-1))

In [37]:
def train_model(encoder_optimizer,decoder_optimizer, encoder, decoder, loss_fun,m_type, dataloader, en_lang,\
                num_epochs=60, val_every = 1, train_bleu_every = 10,clip = 0.1, rm = 0.8, enc_scheduler = None,\
               dec_scheduler = None):
    best_score = 0
    best_bleu = 0
    loss_hist = {'train': [], 'val_train': []}
    bleu_hist = {'train': [], 'validate': []}
    best_encoder_wts = None
    best_decoder_wts = None
#     train_sorted = train.sort_values('en_len', ascending = False)
    phases = ['train','val_train']
#     phases = ['train']
    for epoch in range(num_epochs):
        bs_dict = {'train':128,'validate':1, 'train_val':1,'val_train':128}
#         train_used = shuffle_sorted_batches(train_sorted, bs_dict['train'])
#         train_used = train.iloc[:50]
        train_used = train
        collate_fn_dict = {'train':vocab_collate_func, 'validate':vocab_collate_func_val,\
                           'train_val':vocab_collate_func_val,'val_train':vocab_collate_func}
        transformed_dataset = {'train': Vietnamese(train_used),
                               'validate': Vietnamese(val, val = True),
                               'train_val':Vietnamese(train.iloc[:50], val = True),
                               'val_train':Vietnamese(val)
                                                       }

        dataloader = {x: DataLoader(transformed_dataset[x], batch_size=bs_dict[x], collate_fn=collate_fn_dict[x],
                            shuffle=True, num_workers=0) for x in ['train', 'validate', 'train_val','val_train']}
        for ex, phase in enumerate(phases):
            start = time.time()
            total = 0
            top1_correct = 0
            running_loss = 0
            running_total = 0
            if phase == 'train':
                encoder.train()
                decoder.train()
            else:
                encoder.eval()
                decoder.eval()
            for data in dataloader[phase]:
                encoder_optimizer.zero_grad()
                decoder_optimizer.zero_grad()

                encoder_i = data[0].to(device)
                decoder_i = data[1].to(device)
                src_len = data[2].to(device)
                tar_len = data[3].to(device)
#                 print(encoder_i.shape)
                if phase == 'val_train':                
                    out = encode_decode(encoder,decoder,encoder_i,decoder_i,src_len,tar_len,rand_num=rm,val = True )
                else:
                    out = encode_decode(encoder,decoder,encoder_i,decoder_i,src_len,tar_len,rand_num=rm,val = False )
                N = decoder_i.size(0)
#                 out = out.view(-1,en_lang.n_words)
#                 decoder_i = decoder_i.view(-1)
#                 if loss_fun == 'masked_cel':
#                     loss = masked_cross_entropy(out.float(), decoder_i.long(), tar_len)
#                 else:
#                     loss = flatten_cel_loss(out.float(), decoder_i.long(),loss_fun)
                loss = loss_fun(out.float(), decoder_i.long())
                running_loss += loss.item() * N
                
                total += N
                if phase == 'train':
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
                    torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)
                    encoder_optimizer.step()
                    decoder_optimizer.step()
                    
            epoch_loss = running_loss / total 
            loss_hist[phase].append(epoch_loss)
            print("epoch {} {} loss = {}, time = {}".format(epoch, phase, epoch_loss,
                                                                           time.time() - start))
#             if epoch%train_bleu_every ==0:
#                 train_loss, train_bleu_score = validation(encoder,decoder, dataloader['train'],loss_fun, en_lang,max_len,m_type)
#                 bleu_hist['train'].append(train_bleu_score)
#                 print("Train BLEU = ", train_bleu_score)
        if (enc_scheduler is not None) and (dec_scheduler is not None):
            enc_scheduler.step(epoch_loss)
            dec_scheduler.step(epoch_loss)
        if epoch%val_every == 0:
            val_bleu_score = validation_new(encoder,decoder, dataloader['validate'], en_lang, m_type)
            bleu_hist['validate'].append(val_bleu_score)
            print("validation BLEU = ", val_bleu_score)
            if val_bleu_score > best_bleu:
                best_bleu = val_bleu_score
                best_encoder_wts = encoder.state_dict()
                best_decoder_wts = decoder.state_dict()
        print('='*50)
    encoder.load_state_dict(best_encoder_wts)
    decoder.load_state_dict(best_decoder_wts)
    print("Training completed. Best BLEU is {}".format(best_bleu))
    return encoder,decoder,loss_hist,bleu_hist

In [38]:
# def validation_new(encoder, decoder, val_dataloader, lang_en,m_type):
#     encoder.train(False)
#     decoder.train(False)
#     pred_corpus = []
#     true_corpus = []
#     running_loss = 0
#     running_total = 0
#     bl = BLEU_SCORE()
#     for data in val_dataloader:
#         encoder_i = data[0].to(device)
#         src_len = data[2].to(device)
#         bs,sl = encoder_i.size()[:2]
#         en_hid = encoder.initHidden(bs)
#         en_out,en_hid = encoder(encoder_i, en_hid)
#         decoder_hidden = en_hid
#         decoder_input = torch.tensor([[SOS_token]]*bs).to(device)
# #         c = decoder.initHidden(bs)
#         d_out = []
#         for i in range(sl+20):
#             if m_type=="attention":
#                 decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden,en_out)
#             else:
#                 decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden)
#             topv, topi = decoder_output.topk(1)
#             decoder_input = topi.squeeze().detach().view(-1,1)
#             d_out.append(topi.item())
#             if topi.item() == EOS_token:
#                 break
#         d_hid = decoder_hidden
        
#         true_corpus.append(data[-1])
#         pred_sent = convert_id_list_2_sent(d_out,lang_en)
#         pred_corpus.append(pred_sent)
# #         print("True Sentence:",data[-1])
# #         print("Pred Sentence:", pred_sent)
# #         print('-*'*50)
#     score = bl.corpus_bleu(pred_corpus,[true_corpus],lowercase=True)[0]
#     return score

In [39]:
# def validation_new(encoder, decoder, val_dataloader, lang_en,m_type):
#     encoder.train(False)
#     decoder.train(False)
#     pred_corpus = []
#     true_corpus = []
#     running_loss = 0
#     running_total = 0
#     bl = BLEU_SCORE()
#     for data in val_dataloader:
#         encoder_i = data[0].to(device)
#         src_len = data[2].to(device)
#         bs,sl = encoder_i.size()[:2]
#         en_out,en_hid = encoder(encoder_i,src_len)
#         decoder_hidden = en_hid
#         decoder_input = torch.tensor([[SOS_token]]*bs).to(device)
#         c = decoder.initHidden(bs)
#         d_out = []
#         for i in range(sl+20):
#             if m_type=="attention":
#                 decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden,en_out,src_len,c)
#             else:
#                 decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden)
#             topv, topi = decoder_output.topk(1)
#             decoder_input = topi.squeeze().detach().view(-1,1)
#             d_out.append(topi.item())
#             if topi.item() == EOS_token:
#                 break
#         d_hid = decoder_hidden
        
#         true_corpus.append(data[-1])
#         pred_sent = convert_id_list_2_sent(d_out,lang_en)
#         pred_corpus.append(pred_sent)
# #         print("True Sentence:",data[-1])
# #         print("Pred Sentence:", pred_sent)
# #         print('-*'*50)
#     score = bl.corpus_bleu(pred_corpus,[true_corpus],lowercase=True)[0]
#     return score

In [40]:
def validation_new(encoder, decoder, val_dataloader, lang_en,m_type, verbose = False):
    encoder.eval()
    decoder.eval()
    pred_corpus = []
    true_corpus = []
    running_loss = 0
    running_total = 0
    bl = BLEU_SCORE()
    for data in val_dataloader:
        encoder_i = data[0].to(device)
        src_len = data[2].to(device)
        bs,sl = encoder_i.size()[:2]
        en_out,en_hid,en_c = encoder(encoder_i,src_len)
        max_src_len_batch = max(src_len).item()
        prev_hiddens = en_hid
        prev_cs = en_c
        decoder_input = torch.tensor([[SOS_token]]*bs).to(device)
        prev_output = torch.zeros((bs, en_out.size(-1))).to(device)
        d_out = []
        for i in range(sl*2):
            out_vocab, prev_output,prev_hiddens, prev_cs, attention_score = decoder(decoder_input,prev_output, \
                                                                                    prev_hiddens,prev_cs, en_out,\
                                                                                    src_len)
            topv, topi = out_vocab.topk(1)
#             decoder_input = topi.squeeze().detach().view(-1,1)
            d_out.append(topi.item())
            decoder_input = topi.squeeze().detach().view(-1,1)
            if topi.item() == EOS_token:
                break
        
        true_corpus.append(data[-1])
        pred_sent = convert_id_list_2_sent(d_out,lang_en)
        pred_corpus.append(pred_sent)
        if verbose:
            print("True Sentence:",data[-1])
            print("Pred Sentence:", pred_sent)
            print('-*'*50)
    score = bl.corpus_bleu(pred_corpus,[true_corpus],lowercase=True)[0]
    return score

In [41]:
def masked_cross_entropy(logits, target, length):
    logits = logits.transpose(1,2)
#     print("logits",logits.shape)
#     print("target",target.shape)
    # logits_flat: (batch * max_len, num_classes)
    logits_flat = logits.contiguous().view(-1, logits.size(-1))
    # log_probs_flat: (batch * max_len, num_classes)
#     log_probs_flat = F.log_softmax(logits_flat, dim = 1)
    # target_flat: (batch * max_len, 1)
    target_flat = target.view(-1, 1)
#     print("tar_flat", target_flat.shape)
    # losses_flat: (batch * max_len, 1)
    losses_flat = -torch.gather(logits_flat, dim=1, index=target_flat)
    # losses: (batch, max_len)
    losses = losses_flat.view(*target.size())
    # mask: (batch, max_len)
    mask = sequence_mask(sequence_length=length, max_len=target.size(1))
    losses = losses * mask
    loss = losses.sum() / length.float().sum()
    return loss

In [42]:
criterion = 'masked_cel'

# Old Interesting performance

In [305]:
criterion = nn.NLLLoss(ignore_index=PAD_IDX)

In [77]:
encoder = EncoderRNN(vi_lang.n_words,512,512, 2).to(device)
decoder = AttentionDecoderRNN(en_lang.n_words,512,1024,n_layers=2).to(device)

In [78]:
# encoder = OldEncoderRNN(vi_lang.n_words,512,bi = True).to(device)
# decoder = OldAttentionDecoderRNN(512,en_lang.n_words, True, MAX_LEN, attention_type='t2').to(device)

In [79]:
encoder.rnn

LSTM(512, 512, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)

In [80]:
encoder_optimizer = optim.Adam(encoder.parameters(), lr=1e-4)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=1e-4)

In [81]:
enc, dec, loss_hist, acc_hist = train_model(encoder_optimizer, decoder_optimizer, encoder, decoder, criterion,\
                                            "attention", dataloader,en_lang, num_epochs = 50)

epoch 0 train loss = 5.654132865225413, time = 341.1948525905609
validation BLEU =  0.0
epoch 1 train loss = 5.45065745284289, time = 341.9193711280823
validation BLEU =  0.0
epoch 2 train loss = 5.210996340336155, time = 342.40592312812805
validation BLEU =  0.0
epoch 4 train loss = 4.809566239472573, time = 341.6780881881714
validation BLEU =  0.0
epoch 5 train loss = 4.63142532622266, time = 341.56193590164185
validation BLEU =  0.0
epoch 6 train loss = 4.520703522670287, time = 341.8052592277527
validation BLEU =  0.007257456178771249
epoch 7 train loss = 4.4197971115632395, time = 341.1301064491272
validation BLEU =  0.00778988176186667
epoch 8 train loss = 4.318348191080374, time = 341.3023076057434
validation BLEU =  0.00873574484162468
epoch 9 train loss = 4.237790871660339, time = 341.7362952232361
validation BLEU =  0.01489571311812371
epoch 10 train loss = 4.151050939980724, time = 341.4068603515625
validation BLEU =  0.019931957373692067
epoch 11 train loss = 4.091922725855

In [84]:
validation_new(enc,dec,dataloader['train_val'],en_lang, 'attention', verbose=True)

True Sentence: Rachel Pike : The science behind a climate headline
Pred Sentence: UKN UKN revolution .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: In 4 minutes , atmospheric chemist Rachel Pike provides a glimpse of the massive scientific effort behind the bold headlines on climate change , with her team -- one of thousands who contributed -- taking a risky flight over the rainforest in pursuit of data on a key molecule .
Pred Sentence: In four minutes , the UKN UKN UKN UKN UKN of of of the the the the the the the , , , , the the the , , , , , , , , , , the the the the the the the the the the the
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: I &apos;d like to talk to you today about the scale of the scientific effort that goes into making the headlines you see in the paper .
Pred Sentence: I want you to know about the great challenges of the chal

True Sentence: And it takes weeks to perform our integrations .
Pred Sentence: It &apos;s takes a week of weeks .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: And we perform dozens of integrations in order to understand what &apos;s happening .
Pred Sentence: We need hundreds of these kinds of algorithms to do that .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: We also fly all over the world looking for this thing .
Pred Sentence: We &apos;re around around the world to to .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: I recently joined a field campaign in Malaysia . There are others .
Pred Sentence: I recently joined a UKN in the UKN . . . . . . .
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
True Sentence: We found a glob

6.89490079655882

# Second with shuffled batches

In [43]:
criterion = nn.NLLLoss(ignore_index=PAD_IDX)

In [44]:
encoder = EncoderRNN(vi_lang.n_words,512,512, 2).to(device)
decoder = AttentionDecoderRNN(en_lang.n_words,512,1024,n_layers=2).to(device)

In [45]:
# encoder = OldEncoderRNN(vi_lang.n_words,512,bi = True).to(device)
# decoder = OldAttentionDecoderRNN(512,en_lang.n_words, True, MAX_LEN, attention_type='t2').to(device)

In [46]:
# encoder_optimizer = optim.Adam(encoder.parameters(), lr = 5e-3)
# decoder_optimizer = optim.Adam(decoder.parameters(), lr = 5e-3)
encoder_optimizer = optim.SGD(encoder.parameters(), lr=0.25,nesterov=True, momentum = 0.99)
enc_scheduler = ReduceLROnPlateau(encoder_optimizer, min_lr=1e-4,  patience=0)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=0.25,nesterov=True, momentum = 0.99)
dec_scheduler = ReduceLROnPlateau(decoder_optimizer, min_lr=1e-4,  patience=0)

In [ ]:
enc, dec, loss_hist, acc_hist = train_model(encoder_optimizer, decoder_optimizer, encoder, decoder, criterion,\
                                            "attention", dataloader,en_lang, num_epochs = 15, rm = 0.95,\
                                           enc_scheduler = enc_scheduler, dec_scheduler = dec_scheduler)

epoch 0 train loss = 5.448357953040741, time = 663.8662695884705
epoch 0 val_train loss = 6.205217158380743, time = 3.224254846572876
validation BLEU =  1.2640656972990225
epoch 1 train loss = 4.466191016466267, time = 664.2587950229645
epoch 1 val_train loss = 6.539027952621412, time = 3.2456345558166504
validation BLEU =  3.1767704213891608
epoch 2 train loss = 4.064321100128371, time = 664.3784079551697
epoch 2 val_train loss = 5.843420616835826, time = 3.242091417312622
validation BLEU =  4.054449644495632
epoch 3 train loss = 3.8865785588489707, time = 664.5217673778534
epoch 3 val_train loss = 5.926745017620291, time = 3.223454713821411
validation BLEU =  5.074489796898661
epoch 4 train loss = 3.735492187864453, time = 664.6877009868622
epoch 4 val_train loss = 5.5243214929141455, time = 3.2586846351623535
validation BLEU =  5.5365409862525725
epoch 5 train loss = 3.663915344385771, time = 666.0000283718109
epoch 5 val_train loss = 5.482947348044122, time = 3.227259397506714
vali

In [ ]:
validation_new(enc,dec,dataloader['train_val'],en_lang, 'attention', verbose=True)